# **FOREWORD**

This is a starter kernel for the [Playground Season-5 Episode 7](https://www.kaggle.com/competitions/playground-series-s5e7) multiclass competition with accuracy metric. This needs to be maximized. Since accuracy is not a smooth metric and does not climb well, we use log-loss proxy while training the model. 

We start off with simple EDA and ML models and then improve the results as time passes! Let's get started! <br>
In this version, I use a simple offline ML models with 5-folds and make a submission. Let's see how this goes!

#### **NOTE**
I have simplified my ModelTrainer class for the concurrent playground episode. Please peruse it and provide me relevant feedback.

# **IMPORTS**

In [1]:
!uv pip install -q --system -r /kaggle/input/playgrounds5e7-public-imports-v1/req_kaggle.txt

season  = 5
episode = 7

exec( open(f"/kaggle/input/playgrounds{season}e{episode}-public-imports-v1/myimports.py", "r").read() )
exec( open(f"/kaggle/input/playgrounds{season}e{episode}-public-imports-v1/myutils.py", "r").read() )
exec( open(f"/kaggle/input/playgrounds{season}e{episode}-public-imports-v1/training.py", "r").read() )
exec( open(f"/kaggle/input/playgrounds{season}e{episode}-public-imports-v1/mypp.py", "r").read() )

print()

---> Imports- part 1 done
---> Sklearn = 1.7.0 | Pandas = 2.2.3 | Polars = 1.31.0
---> Commencing imports-part2
---> XGBoost = 3.0.2 | LightGBM = 4.6.0
---> Imports- part 2 done
---> Seeding everything

---> Imports done




# **CONFIGURATION**

In [2]:
%%time

class CFG:
    """
    Configuration class for parameters and CV strategy for tuning and training
    Some parameters may be unused here as this is a general configuration class
    """

    # Data preparation:-
    version_nb         = 1
    model_id           = "V1_1"
    model_label        = "ML"
    test_req           = False
    test_iter          = 200
    gpu_switch         = "ON" if torch.cuda.is_available() else "OFF"
    state              = 42
    target             = f"Personality"
    grouper            = f""
    
    tgt_mapper         = {"Extrovert" : 0, "Introvert" : 1}
    
    ip_path            = f"/kaggle/input/playground-series-s5e7"
    op_path            = f"/kaggle/working"
    orig_path          = f"/kaggle/input/extrovert-vs-introvert-behavior-data/personality_dataset.csv"
    data_path          = f""
    dtl_preproc_req    = True
    ftre_plots_req     = False
    ftre_imp_req       = False
    nb_orig            = 0
    orig_all_folds     = True

    # Model Training:-
    pstprcs_oof        = False
    pstprcs_train      = False
    pstprcs_test       = False
    ML                 = True
    test_preds_req     = True
    n_splits           = 5
    n_repeats          = 1
    nbrnd_erly_stp     = 0
    mdlcv_mthd         = 'SKF'
    metric_obj         = 'maximize'

    # Global variables for plotting:-
    grid_specs = {'visible'  : True,
                  'which'    : 'both',
                  'linestyle': '--',
                  'color'    : 'lightgrey',
                  'linewidth': 0.75
                 }

    title_specs = {'fontsize'   : 9,
                   'fontweight' : 'bold',
                   'color'      : '#992600',
                  }

cv_selector = \
{
 "RKF"   : RepeatedKFold(n_splits = CFG.n_splits, n_repeats= CFG.n_repeats, random_state= CFG.state),
 "RSKF"  : RepeatedStratifiedKFold(n_splits  = CFG.n_splits, n_repeats= CFG.n_repeats, random_state= CFG.state),
 "SKF"   : StratifiedKFold(n_splits = CFG.n_splits, shuffle = True, random_state= CFG.state),
 "KF"    : KFold(n_splits = CFG.n_splits, shuffle = True, random_state= CFG.state),
 "GKF"   : GroupKFold(n_splits = CFG.n_splits)
}

collect()

CPU times: user 240 ms, sys: 767 µs, total: 241 ms
Wall time: 240 ms


0

## **KEY CONFIG PARAMETERS**

|Configuration parameter| Explanation| Data type| Sample values |  
| ---------------------- | ------------------------------- | --------------------- | --------------- |
| version_nb    | Version Number | int | 1 | 
| model_id      | Model ID    | string | V1_1 | 
| model_label   | Model Label | string | ML | 
| test_req      | Test Required| bool | True / False | 
| test_iter     | Test case iterations for models | int | 50 |
| gpu_switch      | Do we need GPU support | bool | True / False |
| state           | Random state | int | 42 |
| target          | Target column | str |  |
| grouper         | CV grouper column | str |  |
| ip_path, op_path | Data paths  | str | |
| pstprcs_* | Do we need post-processing  | bool |True / False |
| ML| Do we need machine learning models  | bool |True / False |
| test_preds_req| Do we need test set predictions (training in inference kernel)  | bool |True / False |
| n_splits/ n_repeats | N-splits and repeats for CV scheme | int | 3/5/10|
| nbrnd_erly_stp | Early stopping rounds | int | 40|
| mdlcv_mthd | Model CV method | str | RSKF|
| ensemble_req | Do we need ensemble | bool | True / False |
| metric_obj   | Metric direction | str | minimize/ maximize |

# **PREPROCESSING**

In [3]:
%%time 

pp = Preprocessor()
pp.DoPreprocessing();

Data shapes - train-test-original | (18524, 8) (6175, 7) (2512, 8)

Train set head


,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
id,,,,,,,,
0,0.000,No,6.000,4.000,No,15.000,5.000,Extrovert
1,1.000,No,7.000,3.000,No,10.000,8.000,Extrovert
2,6.000,Yes,1.000,0.000,nan,3.000,0.000,Introvert
3,3.000,No,7.000,3.000,No,11.000,5.000,Extrovert
4,1.000,No,4.000,4.000,No,13.000,nan,Extrovert



Test set head


,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
id,,,,,,,
18524,3.000,No,7.000,4.000,No,6.000,nan
18525,nan,Yes,0.000,0.000,Yes,5.000,1.000
18526,3.000,No,5.000,6.000,No,15.000,9.000
18527,3.000,No,4.000,4.000,No,5.000,6.000
18528,9.000,Yes,1.000,2.000,Yes,1.000,1.000



Original set head


...


Original information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Time_spent_Alone           2451 non-null   float64
 1   Stage_fear                 2439 non-null   object 
 2   Social_event_attendance    2451 non-null   float64
 3   Going_outside              2447 non-null   float64
 4   Drained_after_socializing  2461 non-null   object 
 5   Friends_circle_size        2437 non-null   float64
 6   Post_frequency             2449 non-null   float64
 7   Personality                2512 non-null   object 
 8   Source                     2512 non-null   object 
dtypes: float64(5), object(4)
memory usage: 176.8+ KB


None


Unique and null values



,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Source
Train_Nunq,12,2,11,8,2,16,11,1
Test_Nunq,12,2,11,8,2,16,11,1
Original_Nunq,12,2,11,8,2,16,11,1
Train_Nulls,"1,190","1,893","1,180","1,466","1,149","1,054","1,264",0
Test_Nulls,425,598,397,466,432,350,408,0
Original_Nulls,61,73,61,65,51,75,63,0



We are using the competition training data only
CPU times: user 909 ms, sys: 13.2 ms, total: 922 ms
Wall time: 967 ms


## **ADVERSARIAL CV**

Adversarial cross-validation is an effective and easy check to see if the test set differs from the train-set. <br>
We do the below steps here and infer this -
1. Consider any classifier as a base model, I prefer any boosted tree model as I don't have to focus too much on preprocessing
2. Load the train and test set features
3. Make a new target column with 1 for test set occurrances and 0 for train-set
4. Classify to predict the test set instances with the features and new target from the step above
5. If the AUC score hovers around 50% (random model), then we can be sure that the train and test set have similar distributions and the model is likely to generalize. Else, if our model is able to differentiate between the train and test data, then our model is unlikely to generalize as-is and further adjustments may be necesary

In [4]:
%%time 

if CFG.dtl_preproc_req == True :
    advcv = AdversarialCVMaker()
    advcv.make_cv(pp.train[pp.test.columns], pp.test)


---> Overall adversarial CV score = 0.5007
---> Train-test distributions are similar

CPU times: user 2.15 s, sys: 20.7 ms, total: 2.17 s
Wall time: 1.13 s


## **NOTES**

- All features seem to be categorical
- Few nulls are present - we can easily deal with them by making a separate category / imputation
- Train-test data appear to be similar in distribution

# **FEATURE CREATION**

Let's start with some n-grams and use the original data as rows

In [5]:
%%time 

Xtrain    = pp.train.drop(CFG.target, axis=1)
Xtest     = pp.test.copy()
ytrain    = pp.train[CFG.target]
PrintColor(f"---> Shapes = {Xtrain.shape} {Xtest.shape}")

num_cols         = Xtrain.select_dtypes(np.number).columns
Xtrain[num_cols] = Xtrain[num_cols].fillna(-1).astype(np.int16).astype("string")
Xtest[num_cols]  = Xtest[num_cols].fillna(-1).astype(np.int16).astype("string")

Xtrain[pp.cat_cols] = Xtrain[pp.cat_cols].astype("string").fillna("missing")
Xtest[pp.cat_cols]  = Xtest[pp.cat_cols].astype("string").fillna("missing")

for col1, col2 in combinations(list(pp.strt_ftre[0:-1]), 2) :
    label = f"{col1}-{col2}"
    Xtrain[label] = Xtrain[col1] + "-" + Xtrain[col2]
    Xtest[label]  = Xtest[col1]  + "-" + Xtest[col2]

print(f"---> Shapes = {Xtrain.shape} {Xtest.shape} | 2-grams")

for col1, col2, col3 in combinations(list(pp.strt_ftre[0:-1]), 3) :
    label = f"{col1}-{col2}-{col3}"
    Xtrain[label] = Xtrain[col1] + "-" + Xtrain[col2] + Xtrain[col3] 
    Xtest[label]  = Xtest[col1]  + "-" + Xtest[col2]  + Xtest[col3] 

print(f"---> Shapes = {Xtrain.shape} {Xtest.shape} | 3-grams")

cat_cols = Xtrain.columns.tolist()
with np.printoptions(linewidth = 150, threshold = 1000):
    PrintColor(f"\n\n---> Category columns")
    pprint(np.array(cat_cols))

_ = utils.CleanMemory()
print()


---> Shapes = (18524, 8) (6175, 8)
---> Shapes = (18524, 29) (6175, 29) | 2-grams
---> Shapes = (18524, 64) (6175, 64) | 3-grams


---> Category columns
array(['Time_spent_Alone', 'Stage_fear', 'Social_event_attendance', 'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
       'Post_frequency', 'Source', 'Time_spent_Alone-Stage_fear', 'Time_spent_Alone-Social_event_attendance', 'Time_spent_Alone-Going_outside',
       'Time_spent_Alone-Drained_after_socializing', 'Time_spent_Alone-Friends_circle_size', 'Time_spent_Alone-Post_frequency',
       'Stage_fear-Social_event_attendance', 'Stage_fear-Going_outside', 'Stage_fear-Drained_after_socializing', 'Stage_fear-Friends_circle_size',
       'Stage_fear-Post_frequency', 'Social_event_attendance-Going_outside', 'Social_event_attendance-Drained_after_socializing',
       'Social_event_attendance-Friends_circle_size', 'Social_event_attendance-Post_frequency', 'Going_outside-Drained_after_socializing',
       'Going_outside-

# **CV-SCHEME**

In [6]:
%%time 

# Initializing the cv scheme:-
cv = cv_selector[CFG.mdlcv_mthd]

if CFG.nb_orig > 0:
    all_df = []
    
    for mysource in ["Competition", "Original"]:
        df = pd.concat([Xtrain.loc[Xtrain.Source == mysource], ytrain], axis=1, join = "inner")
        df.index = range(len(df))
        for fold_nb, (_, dev_idx) in enumerate(cv.split(df, df[CFG.target])):
            df.loc[dev_idx, "fold_nb"] = fold_nb
            
        all_df.append(df)      
    ygrp = pd.concat(all_df, axis=0, ignore_index = True)["fold_nb"].astype(np.uint8)
                      
else:
    df = Xtrain.loc[Xtrain.Source == "Competition"]
    df.index = range(len(df))
    
    for fold_nb, (_, dev_idx) in enumerate(cv.split(df, ytrain.iloc[df.index])):
        df.loc[dev_idx, "fold_nb"] = fold_nb 
    ygrp = df["fold_nb"].astype(np.uint8)

PrintColor(
    f"\n---> Shapes = {Xtrain.shape} {Xtest.shape} {ytrain.shape} {ygrp.shape}"
)


---> Shapes = (18524, 64) (6175, 64) (18524,) (18524,)
CPU times: user 38.5 ms, sys: 11.6 ms, total: 50.2 ms
Wall time: 49.5 ms


# **MODEL TRAINING**

In [7]:
%%time 

Mdl_Master = \
{     
 f'XGB1C'  : [Pipeline(
                 steps = [("TE", TargetEncoder(random_state = CFG.state)),
                          ("M" , XGBC(**{  "objective"            : "binary:logistic",
                                           "eval_metric"          : "logloss",
                                           'device'               : "cuda:0" if CFG.gpu_switch == "ON" else "cpu",
                                           'learning_rate'        : 0.02,
                                           'n_estimators'         : 1500,
                                           'max_depth'            : 5,
                                           'colsample_bytree'     : 0.45,
                                           'verbosity'            : 0,
                                           'random_state'         : CFG.state,
                                           'early_stopping_rounds': None,
                                           'enable_categorical'   : True,
                                          } 
                                       ),
                          )
                         ]
     
              ), 
              {"M__verbose" : 0}
             ],

 f"CB1C"   : [Pipeline(
                 steps = [("TE", TargetEncoder(random_state = CFG.state)),
                          ("M" , CBC(**{   "loss_function"        : "Logloss",
                                           'task_type'            : "GPU" if CFG.gpu_switch == "ON" else "CPU",
                                           'learning_rate'        : 0.02,
                                           'iterations'           : 1500,
                                           'max_depth'            : 5,
                                           'colsample_bylevel'    : 0.45 if CFG.gpu_switch == "OFF" else None,
                                           'verbose'              : 0,
                                           'random_state'         : CFG.state,
                                           'early_stopping_rounds': None,
                                        } 
                                    ),
                          )
                         ]
              ), 
              {"M__verbose" : 0}
             ], 

 f'LGBM1C' : [Pipeline(
                 steps = [("TE", TargetEncoder(random_state = CFG.state)),
                          ("M" , LGBMC(**{ "objective"            : "binary",
                                           "metric"               : "logloss",
                                           'device'               : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                                           'learning_rate'        : 0.02,
                                           'n_estimators'         : 1500,
                                           'max_depth'            : 5,
                                           'colsample_bytree'     : 0.45,
                                           'verbosity'            : -1,
                                           'random_state'         : CFG.state,
                                           'reg_lambda'           : 1.50,
                                          } 
                                       ),
                          )
                         ]
              ), 
              {"M__callbacks" : [log_evaluation(0)]}
             ],

 f'RF1C'  : [Pipeline(
                 steps = [("TE", TargetEncoder(random_state = CFG.state)),
                          ("M" , RFC(**{"n_estimators" : 100,
                                        "max_depth"    : 6,
                                        "min_samples_leaf" : 16, 
                                       } 
                                    ),
                          )
                         ]
              ), 
              {},
             ],

 f'HGB1C' : [Pipeline(
                 steps = [("TE", TargetEncoder(random_state = CFG.state)),
                          ("M" , HGBC(**{'learning_rate'    : 0.03,
                                         'min_samples_leaf' : 12,
                                         'max_iter'         : 500,
                                         'max_depth'        : 5,
                                         'l2_regularization': 0.75,
                                        } 
                                    ),
                          )
                         ]
              ), 
              {},
             ],
}

# Initializing model outputs
OOF_Preds    = {}
Mdl_Preds    = {}

CPU times: user 1.14 ms, sys: 0 ns, total: 1.14 ms
Wall time: 2.23 ms


In [8]:
%%time 

drop_cols = ["Source", "id", "Id", "Label", CFG.target, "fold_nb"]
ytrain_   = ytrain.map(CFG.tgt_mapper).astype(np.uint8)

for method, (mymodel, fit_params) in tqdm(Mdl_Master.items()) :
    md = ModelTrainer(
        drop_cols    = drop_cols, 
        problem_type = "binary",
        len_train    = Xtrain.loc[Xtrain.Source == "Competition"].shape[0]
    )

    _, oof_preds, mdl_preds = \
    md.fit_predict(
        Xtrain.copy(), 
        ytrain_,
        Xtest.copy(), 
        ygrp,
        extra    = None,
        method   = method,
        mymodel  = mymodel,
        cat_cols = None,
        **fit_params,
    )   

    OOF_Preds[method] = oof_preds.flatten()
    Mdl_Preds[method] = mdl_preds.flatten()
    collect();

    score = utils.ScoreMetric(ytrain_.values[0 : len(oof_preds)], oof_preds)
    PrintColor(
        f"---> Overall score = {score :,.8f} | Competition metric",
        color = Fore.CYAN
    )

_ = utils.CleanMemory()
print()

  0%|          | 0/5 [00:00<?, ?it/s]


 XGB1C offline model training



XGB1C: 0it [00:00, ?it/s]

---> OOF score = 0.14981627 | Fold 1
---> OOF score = 0.16376571 | Fold 2
---> OOF score = 0.17338951 | Fold 3
---> OOF score = 0.14728551 | Fold 4
---> OOF score = 0.14486112 | Fold 5

---> Overall score = 0.15582362 +- 0.01096176
---> Overall score = 0.96906716 | Competition metric

 CB1C offline model training



CB1C: 0it [00:00, ?it/s]

...

---> OOF score = 0.14829308 | Fold 1
---> OOF score = 0.16381552 | Fold 2
---> OOF score = 0.17180226 | Fold 3
---> OOF score = 0.14587502 | Fold 4
---> OOF score = 0.14250741 | Fold 5

---> Overall score = 0.15445866 +- 0.01133911
---> Overall score = 0.96895919 | Competition metric

 RF1C offline model training



RF1C: 0it [00:00, ?it/s]

---> OOF score = 0.12711916 | Fold 1
---> OOF score = 0.13451567 | Fold 2
---> OOF score = 0.14004985 | Fold 3
---> OOF score = 0.12633869 | Fold 4
---> OOF score = 0.11987061 | Fold 5

---> Overall score = 0.12957880 +- 0.00699732
---> Overall score = 0.96922911 | Competition metric

 HGB1C offline model training



HGB1C: 0it [00:00, ?it/s]

---> OOF score = 0.13031149 | Fold 1
---> OOF score = 0.13591122 | Fold 2
---> OOF score = 0.14402648 | Fold 3
---> OOF score = 0.12912273 | Fold 4
---> OOF score = 0.12468366 | Fold 5

---> Overall score = 0.13281112 +- 0.00665286
---> Overall score = 0.96874325 | Competition metric

CPU times: user 4min, sys: 26.7 s, total: 4min 27s
Wall time: 1min 49s


# **ENSEMBLE**

We use a simple logistic regression and blend all our single models 

In [9]:
%%time 

drop_cols = ["Source", "id", "Id", "Label", CFG.target, "fold_nb"]
ytrain_   = ytrain.map(CFG.tgt_mapper).astype(np.uint8)
oof_preds = pd.DataFrame(OOF_Preds).assign(Source = "Competition")
mdl_preds = pd.DataFrame(Mdl_Preds).assign(Source = "Competition")

method     = "LR1C"
mymodel    = LRC(C = 0.01, max_iter = 10000)
fit_params = {}

md = ModelTrainer(
    drop_cols    = drop_cols, 
    problem_type = "binary",
    len_train    = oof_preds.shape[0]
)

_, oof_ens_preds, test_preds = \
md.fit_predict(
    oof_preds, 
    ytrain_,
    mdl_preds, 
    ygrp,
    extra    = None,
    method   = method,
    mymodel  = mymodel,
    cat_cols = None,
    **fit_params,
)   

score = utils.ScoreMetric(ytrain_.values[0 : len(oof_ens_preds)], oof_ens_preds)
PrintColor(
    f"---> Overall score = {score :,.8f} | Competition metric",
    color = Fore.CYAN
)

_ = utils.CleanMemory()
print()


 LR1C offline model training



LR1C: 0it [00:00, ?it/s]

---> OOF score = 0.13255909 | Fold 1
---> OOF score = 0.14051231 | Fold 2
---> OOF score = 0.14366660 | Fold 3
---> OOF score = 0.13427436 | Fold 4
---> OOF score = 0.12812539 | Fold 5

---> Overall score = 0.13582755 +- 0.00558241
---> Overall score = 0.96917512 | Competition metric

CPU times: user 325 ms, sys: 11.9 ms, total: 337 ms
Wall time: 330 ms


# **CLOSURE**

In [10]:
%%time 

pp.sub_fl[CFG.target] = np.where(test_preds >= 0.50, 1, 0)
pp.sub_fl[CFG.target] = pp.sub_fl[CFG.target].map({v:k for k, v in CFG.tgt_mapper.items()})

pp.sub_fl.to_csv("submission.csv", index = None)

!ls submission.csv
!head submission.csv

submission.csv
id,Personality
18524,Extrovert
18525,Introvert
18526,Extrovert
18527,Extrovert
18528,Introvert
18529,Extrovert
18530,Extrovert
18531,Introvert
18532,Extrovert
CPU times: user 14.2 ms, sys: 42.9 ms, total: 57.1 ms
Wall time: 313 ms
